<a href="https://colab.research.google.com/github/zzeeddd95/capstone-project/blob/master/SVR%2Blinearreggoogle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
import pandas_datareader as web


/usr/local/lib/python3.6/dist-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [ ]:
#Retrieve stock data
goog = web.DataReader('GOOG', data_source='yahoo', start='2010-05-27', end='2020-07-06')
print(goog.head())

                  High         Low  ...     Volume   Adj Close
Date                                ...                       
2010-05-27  245.235794  239.626816  ...  5639200.0  244.314255
2010-05-28  245.803665  240.598175  ...  5811200.0  241.908264
2010-06-01  244.613129  239.163559  ...  5353500.0  240.284363
2010-06-02  246.012878  239.831055  ...  5100500.0  245.763824
2010-06-03  253.051498  246.426331  ...  7328700.0  251.855988

[5 rows x 6 columns]


In [ ]:
#Filter out data to only obtain close price (independant variable)
goog = goog[['Close']]
goog

,Close
Date,
2010-05-27,244.314255
2010-05-28,241.908264
2010-06-01,240.284363
2010-06-02,245.763824
2010-06-03,251.855988
...,...
2020-06-29,1394.969971
2020-06-30,1413.609985
2020-07-01,1438.040039


In [ ]:
#Create a variable to allow for predicting 'n days' into the future
future_out =30

In [ ]:
#Create the dependant variable which will be shifted 'n' units up
goog['Prediction'] = goog[['Close']].shift(-future_out)
goog.tail()

,Close,Prediction
Date,,
2020-06-29,1394.969971,NaN
2020-06-30,1413.609985,NaN
2020-07-01,1438.040039,NaN
2020-07-02,1464.699951,NaN
2020-07-06,1495.699951,NaN


In [ ]:
#create independant data set by converting the dataframe to a numpy array
googdf = np.array(goog.drop(['Prediction'],1))
#Renove the last 'n' rows
googdfx = googdf[:-future_out] #therefore for every column, all of the rows will displayed - future out
googdfx

array([[ 244.31425476],
       [ 241.90826416],
       [ 240.28436279],
       ...,
       [1373.48498535],
       [1406.7199707 ],
       [1402.80004883]])

In [ ]:
#creat dependant dataset (prediction), all values including any Nan's
googdfy = np.array(goog['Prediction'])
#Filter this data by obtaining all values except the last 'n' rows
googdfy = googdfy[:-future_out]
googdfy

array([ 237.02656555,  243.68659973,  244.75260925, ..., 1438.04003906,
       1464.69995117, 1495.69995117])

In [ ]:
#split data into 80% training and 20%testing
googdfx_train , googdfx_test, googdfy_train, googdfy_test = train_test_split(googdfx, googdfy, test_size = 0.2)

In [ ]:
#Create and train the SVR model
SVRgoog = SVR(kernel='rbf', C=1e3,gamma=0.1)
SVRgoog.fit(googdfx_train, googdfy_train)

SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
#test model accuracy via score which returns the coefficient of determination (R^2) of prediction
SVMconfidence = SVRgoog.score(googdfx_test, googdfy_test)
print("SVM confidence", SVMconfidence)

SVM confidence 0.9601338419246367


In [ ]:
#Create and train the linear regression model
linear = LinearRegression()
#train the model
linear.fit(googdfx_train, googdfy_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
#test model accuracy via score which returns the coefficient of determination (R^2) of prediction
linearconfidence = linear.score(googdfx_test, googdfy_test)
print("Linear regression confidence", linearconfidence)

Linear regression confidence 0.9732784640588382


In [ ]:
#set goog_forecast = to the last 30 rows of original dataset from close price column
goog_forecast = np.array(goog.drop(['Prediction'],1))[-future_out:]
print(goog_forecast)


[[1410.42004395]
 [1417.02001953]
 [1417.83996582]
 [1416.72998047]
 [1428.92004395]
 [1431.81994629]
 [1439.2199707 ]
 [1436.38000488]
 [1412.18005371]
 [1438.39001465]
 [1446.60998535]
 [1456.16003418]
 [1465.84997559]
 [1403.83996582]
 [1413.18005371]
 [1419.84997559]
 [1442.7199707 ]
 [1451.11999512]
 [1435.95996094]
 [1431.7199707 ]
 [1451.85998535]
 [1464.41003418]
 [1431.9699707 ]
 [1441.32995605]
 [1359.90002441]
 [1394.9699707 ]
 [1413.60998535]
 [1438.04003906]
 [1464.69995117]
 [1495.69995117]]


In [ ]:
#Print the SVR predicted values for the upcoming 'n' days
SVRprediction = SVRgoog.predict(goog_forecast)
print(SVRprediction)

[ 994.28127854 1150.37190333 1281.59205258 1105.70485538 1346.44433221
 1257.58359939 1393.16738926 1169.60803289  884.67191625 1368.24128893
 1080.45620687 1213.03126656  855.90304896 1477.84532952  871.65963578
 1420.52170591 1038.61494431 1160.98049975 1134.04596731 1268.05833595
 1190.60214476  860.86682824 1241.84345208 1201.93512897 1512.05175325
 1495.41677905  874.04688364 1341.38879282  859.14906935  855.21290938]


In [ ]:
#Print the linear regression predicted values for the upcoming 'n' days
linearprediction = linear.predict(goog_forecast)
print(linearprediction)

[1423.05695783 1429.64166349 1430.45971272 1429.35229547 1441.51415561
 1444.40734864 1451.79025212 1448.95685693 1424.81289558 1450.96221627
 1459.16316898 1468.6911225  1478.35864493 1416.49210359 1425.81058194
 1432.46507206 1455.2821544  1463.66274424 1448.53778482 1444.30760437
 1464.4010224  1476.92203502 1444.55702596 1453.89535573 1372.6538231
 1407.64263039 1426.23951888 1450.6130504  1477.21128125 1508.13955862]


In [ ]:
#Retrieve stock data
googstock = web.DataReader('GOOG', data_source='yahoo', start='2020-05-28', end='2020-07-05')
googstock = googstock[['Close']]
googstock.head()

,Close
Date,
2020-05-28,1416.729980
2020-05-29,1428.920044
2020-06-01,1431.819946
2020-06-02,1439.219971
2020-06-03,1436.380005
